In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 50.0 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 636 kB 49.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import re

def clean(data):
  tokens = data.split()
  translation_table = str.maketrans('', '', "\"#$%&'()*+-/:;<=>@[\]^_`{|}~?!.,")
  tokens = [w.translate(translation_table) for w in tokens]
  tokens = [word.lower() for word in tokens]

  return ' '.join(tokens)

data = []
labels = []

# read news
with open('/content/drive/MyDrive/dataset/SMSSpamCollection.txt') as f:
    lines = [line.rstrip() for line in f]

for line in lines:
  tmp = line.split('\t')
  data.append(clean(tmp[1]))
  
  if tmp[0]=='ham':
    labels.append(0)
  
  if tmp[0]=='spam':
    labels.append(1)
  

In [ ]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(labels)

In [ ]:
labels[0]

array([1., 0.], dtype=float32)

In [ ]:
from transformers import BertTokenizer
import numpy as np

input_ids=[]
attention_masks=[]

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for sent in data:
    bert_inp = bert_tokenizer.encode_plus(sent, add_special_tokens = True, max_length =64, pad_to_max_length = True, 
                                        return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
len(input_ids),len(attention_masks),len(labels)

(5574, 5574, 5574)

In [ ]:
from sklearn.model_selection import train_test_split

train_inp, val_inp, train_label, val_label, train_mask, val_mask = train_test_split(
   input_ids, labels, attention_masks, test_size=0.20, random_state=1000)

test_inp, val_inp, test_label, val_label, test_mask, val_mask = train_test_split(
   val_inp, val_label, val_mask, test_size=0.5, random_state=1000)

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]
X = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(X)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 4,
                       validation_data = ([val_inp, val_mask],val_label))


Epoch 1/4
140/140 [==============================] - 134s 825ms/step - loss: 0.0826 - accuracy: 0.9715 - val_loss: 0.0277 - val_accuracy: 0.9910
Epoch 2/4
140/140 [==============================] - 112s 804ms/step - loss: 0.0180 - accuracy: 0.9939 - val_loss: 0.0230 - val_accuracy: 0.9928
Epoch 3/4
140/140 [==============================] - 112s 802ms/step - loss: 0.0026 - accuracy: 0.9989 - val_loss: 0.0515 - val_accuracy: 0.9767
Epoch 4/4
140/140 [==============================] - 112s 802ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.0180 - val_accuracy: 0.9928


In [ ]:
import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[1]
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(embeddings)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
outputs (Dense)                 (None, 2)            1538        tf_bert_model_1[0][1]      

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 5,
                       validation_data = ([val_inp, val_mask],val_label))


Epoch 1/5
140/140 [==============================] - 132s 808ms/step - loss: 0.0834 - accuracy: 0.9756 - val_loss: 0.0257 - val_accuracy: 0.9928
Epoch 2/5
140/140 [==============================] - 109s 782ms/step - loss: 0.0272 - accuracy: 0.9935 - val_loss: 0.0083 - val_accuracy: 0.9982
Epoch 3/5
140/140 [==============================] - 109s 781ms/step - loss: 0.0141 - accuracy: 0.9960 - val_loss: 0.0964 - val_accuracy: 0.9785
Epoch 4/5
140/140 [==============================] - 109s 782ms/step - loss: 0.0127 - accuracy: 0.9960 - val_loss: 0.0239 - val_accuracy: 0.9928
Epoch 5/5
140/140 [==============================] - 109s 781ms/step - loss: 8.6104e-04 - accuracy: 1.0000 - val_loss: 0.0256 - val_accuracy: 0.9946
